In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import numpy as np

In [2]:
cd /content/drive/MyDrive/DeepLearing

/content/drive/MyDrive/DeepLearing


In [3]:
def get_dataset_dataloader(root='datasets/cifar10', split='train', transform=None, batch_size=32, shuffle=True):
    if split == 'train':
        trainset = torchvision.datasets.CIFAR10(root=root, train=True,
                                                download=True, transform=transform)

        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                                shuffle=shuffle, num_workers=2)
        return trainset, trainloader
    
    elif split == 'test':
        
        testset = torchvision.datasets.CIFAR10(root=root, train=False,
                                            download=True, transform=transform)
        testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                                shuffle=shuffle, num_workers=2)
        return testset, testloader
    else:
        raise ValueError('Not support split name: {}. Please try train or test.'.format(split))

In [4]:
def get_transform():
    return  torchvision.transforms.Compose(
                [
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # need to change
                ]
            
            )

In [5]:
transform = get_transform()
trainset, trainloader = get_dataset_dataloader(split='train', transform=transform,batch_size=64)
testset, testloader = get_dataset_dataloader(split='test', transform=transform,batch_size=64)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
class iNet(nn.Module):
    def __init__(self):
        super(iNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=1)

        self.conv2 = nn.Conv2d(16, 32, 3)
        self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2)

        self.conv3 = nn.Conv2d(32, 64, 5, 1)
        self.pool3 = nn.MaxPool2d(kernel_size=2,stride=2)

        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 10)
        self.logSoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x1 = self.conv1(x)
        x1 = nn.functional.normalize(x1,dim=3)
        x1 = self.pool1(F.relu(x1))
        x1 = self.dropout(x1)

        x2 = self.conv2(x1)
        x2 = nn.functional.normalize(x2,dim=3)
        x2 = self.pool2(F.relu(x2))
        x2 = self.dropout(x2)


        x3 = self.conv3(x2)
        x3 = nn.functional.normalize(x3,dim=3)
        x3 = self.pool3(F.relu(x3))
        x3 = self.dropout(x3)

        x4 = torch.flatten(x3, 1)

        z = self.fc1(x4)
        z = F.relu(z)
        z = self.dropout(z)

        output = self.fc2(z)       
        output = self.logSoftmax(output)
        return output

In [9]:
from tqdm import tqdm_notebook


# Keep track of losses for plotting


from sklearn.metrics import accuracy_score
def trainEpoch(net,criterion,net_optimizer,train_dataloader,test_dataloader,accTrain,accTest,bestScore):
  
    epochs = 5
    
    # bestScore = 0
    # accTrain = []
    # accTest = []

    for epoch in range(epochs):
        print('-'*40)
        print('Epoch:',epoch)
        net.train()
        y_pred = []
        y_true = []
        
        for i, batch in tqdm_notebook(enumerate(train_dataloader)):
            net.zero_grad()

            input1 = batch[0].to(device)
            

            y = batch[1].to(device)
            output = net(input1)
            loss = criterion(output, y)
            loss.backward()
            net_optimizer.step()

            y_direc = torch.argmax(output,dim = 1)
            y_pred+= y_direc.cpu()
            y_true+=y.cpu()

        
        y_pred = np.array(y_pred)
        y_true = np.array(y_true)
        score = accuracy_score(y_true, y_pred)
        print("Train Accuracy",score)
        accTrain.append(score)

        net.eval()
        with torch.no_grad():
            y_pred = []
            y_true = []
            for i, batch in tqdm_notebook(enumerate(test_dataloader)):
                net.zero_grad()

                input1 = batch[0].to(device)
                
                y = batch[1].to(device)
                output = net(input1)
                
                y_direc = torch.argmax(output,dim = 1)
                y_pred+= y_direc.cpu()
                y_true+=y.cpu()
            score = accuracy_score(y_true, y_pred)
            accTest.append(score)
            print("Test Accuracy",score)
    #         if(score > bestScore):
    #             bestScore = score
    #             torch.save({
    #               'epoch': epoch,
    #               'model_state_dict': net.state_dict(),
    #               'optimizer_state_dict': net_optimizer.state_dict(),
    #               'best_score': bestScore
    #               }, 'weights/model.pth')
    # torch.save({
    #     'acc_train':accTrain,
    #     'acc_test':accTest
    # },'weights/accuracy_score.pth')


    return 0



In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
net = iNet().to(device)
net_optimizer = optim.Adam(lr=0.001,params= net.parameters())

In [11]:
summary(net,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]           1,216
         MaxPool2d-2           [-1, 16, 26, 26]               0
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 32, 24, 24]           4,640
         MaxPool2d-5           [-1, 32, 12, 12]               0
           Dropout-6           [-1, 32, 12, 12]               0
            Conv2d-7             [-1, 64, 8, 8]          51,264
         MaxPool2d-8             [-1, 64, 4, 4]               0
           Dropout-9             [-1, 64, 4, 4]               0
           Linear-10                  [-1, 128]         131,200
          Dropout-11                  [-1, 128]               0
           Linear-12                   [-1, 10]           1,290
       LogSoftmax-13                   [-1, 10]               0
Total params: 189,610
Trainable params:

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [14]:
checkAcc = torch.load('weights/accuracy_score.pth')
accTrain = checkAcc['acc_train']
accTest = checkAcc['acc_test']
# print(accTrain)

In [ ]:
# import matplotlib.pyplot as plt
# plt.plot(accTrain,'r',label='train')
# plt.plot(accTest,'g',label='test')
# plt.ylabel('some numbers')
# plt.legend()
# plt.show()

In [12]:
checkpoin = torch.load('weights/model.pth')
bestScore = checkpoin['best_score']
bestScore

0.6961

In [16]:
net.load_state_dict(checkpoin['model_state_dict'])

<All keys matched successfully>

In [18]:
trainEpoch(net,criterion,net_optimizer,trainloader,testloader,accTrain,accTest,bestScore)

----------------------------------------
Epoch: 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

Train Accuracy 0.75978


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

0it [00:00, ?it/s]

Train Accuracy 0.76052


0it [00:00, ?it/s]

Test Accuracy 0.6947


0